In [1]:
import os
import numpy as np
import platform
import pandas as pd
import cx_Oracle as odb
import matplotlib.pyplot as plt
import pandas.io.sql as psql
import array

In [2]:
# working dir
if platform.system() == 'Windows':
    os.chdir(os.path.join(os.getenv('HOME'),'Documents', 'Development', 'dst', 'examples', 'abc_db'))
else:
    os.chdir(os.path.join(os.getenv('DST'),'examples', 'abc_db'))
print(os.getcwd())

# plots 
%matplotlib inline
np.set_printoptions(precision=4)
plt.rc('figure', figsize=(10, 8))

C:\Users\J28976\Documents\Development\dst\examples\abc_db


In [3]:
# Build connection string
user     = "abc"
sid      = "XE" if platform.system()=='Windows' else "ORCL"
pswd     = "abc"
conn_str =" %s/%s@%s" % (user, pswd, sid)

# Connect to Oracle and test
conn = odb.connect(user, pswd, sid)
if (conn):
    print("Connection successful")
    print(conn.version)
else:
    print("Connection not successful")

#conn.close()

Connection successful
11.2.0.2.0


In [4]:
sql = """\
select p.*, t.name team 
from players p, teams t 
where p.teams_id=t.id\
"""

players_df = psql.read_sql(sql, conn)
players_df.columns = [c.lower() for c in players_df.columns]

In [5]:
players_df

,id,name,starts,goals,image1,image2,image3,teams_id,team
0,1,t1p1,12,7,<cx_Oracle.OBJECT object at 0x0000000017DC0928>,<cx_Oracle.OBJECT object at 0x0000000017DC0960>,<cx_Oracle.OBJECT object at 0x0000000017DC0998>,1,team1
1,2,t1p2,2,0,<cx_Oracle.OBJECT object at 0x0000000017DC09D0>,<cx_Oracle.OBJECT object at 0x0000000017DC0A08>,<cx_Oracle.OBJECT object at 0x0000000017DC0A40>,1,team1
2,3,t1p3,5,11,<cx_Oracle.OBJECT object at 0x0000000017DC0A78>,<cx_Oracle.OBJECT object at 0x0000000017DC0AB0>,<cx_Oracle.OBJECT object at 0x0000000017DC0AE8>,1,team1
3,4,t2p1,5,8,<cx_Oracle.OBJECT object at 0x0000000017DC0B20>,<cx_Oracle.OBJECT object at 0x0000000017DC0B58>,<cx_Oracle.OBJECT object at 0x0000000017DC0B90>,2,team2
4,5,t2p2,0,0,<cx_Oracle.OBJECT object at 0x0000000017DC0BC8>,<cx_Oracle.OBJECT object at 0x0000000017DC0C00>,<cx_Oracle.OBJECT object at 0x0000000017DC0C38>,2,team2
5,6,t2p3,1,0,<cx_Oracle.OBJECT object at 0x0000000017DC0C70>,<cx_Oracle.OBJECT object at 0x0000000017DC0CA8>,<cx_Oracle.OBJECT object at 0x0000000017DC0CE0>,2,team2


In [6]:
obj = players_df.iloc[0].image1
obj

In [7]:
dir(obj)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'type']

In [8]:
obj.type, obj.DTYPE, obj.NDIM, obj.DIM1, obj.DIM2, obj.DIM3, obj.AVG, obj.STD

(<cx_Oracle.ObjectType ABC.ARRAYINFO>,
 'uint8',
 3.0,
 743.0,
 508.0,
 3.0,
 108.590455802715,
 43.8245299379253)

In [15]:
sql = """
SELECT player.id AS player_id,
  player.name,
  player.starts,
  player.goals,
  player.teams_id,
  player.image1.avg,
  player.image1.std,
  player.image1.ndim,
  player.image1.dim1,
  player.image1.dim2,
  player.image1.dim3,
  player.image1.dim4,
  player.image1.dtype,
  player.image1.value
FROM players player
"""
df  = psql.read_sql(sql, conn)
df.columns = [c.lower() for c in df.columns]
df

,player_id,name,starts,goals,teams_id,image1.avg,image1.std,image1.ndim,image1.dim1,image1.dim2,image1.dim3,image1.dim4,image1.dtype,image1.value
0,1,t1p1,12,7,1,108.590456,43.824530,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
1,2,t1p2,2,0,1,108.424222,43.737109,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
2,3,t1p3,5,11,1,108.423766,43.741242,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
3,4,t2p1,5,8,2,108.472497,43.796894,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
4,5,t2p2,0,0,2,108.498438,43.829959,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
5,6,t2p3,1,0,2,108.453271,43.802645,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...


In [32]:
dfs = []
for chunks in pd.read_sql_query(sql, conn, chunksize=2):
    dfs.append(chunk)
f = pd.concat(dfs)
f

,PLAYER_ID,NAME,STARTS,GOALS,TEAMS_ID,IMAGE1.AVG,IMAGE1.STD,IMAGE1.NDIM,IMAGE1.DIM1,IMAGE1.DIM2,IMAGE1.DIM3,IMAGE1.DIM4,IMAGE1.DTYPE,IMAGE1.VALUE
0,5,t2p2,0,0,2,108.498438,43.829959,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
1,6,t2p3,1,0,2,108.453271,43.802645,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
0,5,t2p2,0,0,2,108.498438,43.829959,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
1,6,t2p3,1,0,2,108.453271,43.802645,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
0,5,t2p2,0,0,2,108.498438,43.829959,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...
1,6,t2p3,1,0,2,108.453271,43.802645,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...


In [16]:
image1 = np.empty_like(df['image1.value'])

for i, idx in enumerate(df.index):
    ndim  = df.iloc[i]['image1.ndim' ]
    dim1  = df.iloc[i]['image1.dim1' ]
    dim2  = df.iloc[i]['image1.dim2' ]
    dim3  = df.iloc[i]['image1.dim3' ] 
    dtype = df.iloc[i]['image1.dtype']
    blob  = df.iloc[i]['image1.value']
    image1[i] = np.array(array.array('B', blob.read()), dtype=np.uint8).reshape((dim1, dim2, dim3))
    
df['image1'] = image1
df

,player_id,name,starts,goals,teams_id,image1.avg,image1.std,image1.ndim,image1.dim1,image1.dim2,image1.dim3,image1.dim4,image1.dtype,image1.value,image1
0,1,t1p1,12,7,1,108.590456,43.824530,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
1,2,t1p2,2,0,1,108.424222,43.737109,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
2,3,t1p3,5,11,1,108.423766,43.741242,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
3,4,t2p1,5,8,2,108.472497,43.796894,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
4,5,t2p2,0,0,2,108.498438,43.829959,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
5,6,t2p3,1,0,2,108.453271,43.802645,3,743,508,3,-9999,uint8,(* (* (* (*!)+!)+!)+!)+#+-#+-#+-#+-#+-#+-#+-#...,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."


In [17]:
df.drop('image1.ndim' , axis=1, inplace=True)
df.drop('image1.dim1' , axis=1, inplace=True)
df.drop('image1.dim2' , axis=1, inplace=True)
df.drop('image1.dim3' , axis=1, inplace=True)
df.drop('image1.dim4' , axis=1, inplace=True)
df.drop('image1.dtype', axis=1, inplace=True)
df.drop('image1.value', axis=1, inplace=True)
df

,player_id,name,starts,goals,teams_id,image1.avg,image1.std,image1.dim4,image1
0,1,t1p1,12,7,1,108.590456,43.824530,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
1,2,t1p2,2,0,1,108.424222,43.737109,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
2,3,t1p3,5,11,1,108.423766,43.741242,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
3,4,t2p1,5,8,2,108.472497,43.796894,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
4,5,t2p2,0,0,2,108.498438,43.829959,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."
5,6,t2p3,1,0,2,108.453271,43.802645,-9999,"[[[32, 40, 42], [32, 40, 42], [32, 40, 42], [3..."


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 9 columns):
player_id      6 non-null int64
name           6 non-null object
starts         6 non-null int64
goals          6 non-null int64
teams_id       6 non-null int64
image1.avg     6 non-null float64
image1.std     6 non-null float64
image1.dim4    6 non-null int64
image1         6 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 480.0+ bytes
